<a href="https://colab.research.google.com/github/irfankhan745/Git-to-VS-Join/blob/main/auto_clicker_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def start_clicking(url, selector, clicks, interval, randomness=0.3):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    driver.get(url)
    time.sleep(3)

    logs = []

    for i in range(clicks):
        try:
            element = driver.find_element(By.CSS_SELECTOR, selector)

            driver.execute_script(
                "arguments[0].scrollIntoView({behavior:'smooth',block:'center'});",
                element
            )

            time.sleep(random.uniform(0.2, 0.6))
            element.click()

            delay = interval + random.uniform(-interval*randomness, interval*randomness)

            logs.append({
                "click": i + 1,
                "selector": selector,
                "delay": round(delay, 2)
            })

            time.sleep(delay)

        except Exception as e:
            logs.append({"error": str(e)})
            break

    return logs
